In [29]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import AmlCompute,ComputeInstance
from azure.ai.ml import command
from azure.ai.ml import command, Output
from azure.ai.ml.constants import AssetTypes

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.16.1
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, opencensus-ext-logging, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
try:
    credential=DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential=InteractiveBrowserCredential()

In [5]:
# Handle to Workspace
mlClient=MLClient.from_config(credential=credential)

Found the config file in: /config.json


In [11]:
cpu_compute_target = "aml-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = mlClient.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")

    # Let's create the Azure ML compute object with the intended parameters
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        # Azure ML Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="STANDARD_DS11_V2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=1,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=120,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )

Creating a new cpu compute target...


In [12]:
cpu_cluster=mlClient.compute.begin_create_or_update(compute=cpu_cluster)

In [13]:
# Scale up to two instance
clusterScale=AmlCompute(
    name="aml-cluster",
    max_instances=2
)
mlClient.begin_create_or_update(entity=clusterScale)

In [14]:
cpuCluster=mlClient.compute.get("aml-cluster")

In [19]:
cpuCluster.name,cpuCluster.max_instances

('aml-cluster', 2)

### Create a Script to Train a Model

In [20]:
!mkdir src

In [21]:
!pwd

/mnt/batch/tasks/shared/LS_root/mounts/clusters/ritishadhikaricompute/code/Users/ritishadhikari


In [22]:
!ls

2_Work_With_Compute.ipynb	  azure-ml-labs     compute-setup.sh.amltmp
2_work_with_compute.ipynb.amltmp  compute-setup.sh  src


In [24]:
!cp azure-ml-labs/Labs/04/src/diabetes.csv src/diabetes.csv 

In [25]:
%%writefile src/diabetes-training.py

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,roc_curve

print("Loading Data ...")
diabetes=pd.read_csv("diabetes.csv")

X=diabetes[
    ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure',
    'TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
].values
y=diabetes['Diabetic'].values

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.30, random_state=0)
reg=0.01
print('Training a logistic regression model with regularization rate of', reg)
model=LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# Calculate the Accuracy
yHat=model.predict(X=X_test)
acc=np.average(yHat==y_test)
print(f"Accuracy:{acc}")

# Calculate AUC
yScores=model.predict_proba(X=X_test)
auc=roc_auc_score(y_true=y_test,y_score=yScores[:,1])
print("AUC: "+str(auc))

Writing src/diabetes-training.py


In [27]:
job=command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute=cpu_compute_target,
    display_name="diabetes-train-cluster",
    experiment_name="diabetes-training"
)

returnedJob=mlClient.jobs.create_or_update(job=job)
amlURL=returnedJob.studio_url
print(f"Monitor your Job at: {amlURL}")


Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading src (0.52 MBs): 100%|███████

Monitor your Job at: https://ml.azure.com/runs/ivory_goat_2p3kzy5sc1?wsid=/subscriptions/18a1f27f-edf5-495e-9acb-753c93335294/resourcegroups/rg-dp100-labs/workspaces/mlw-dp100-labs&tid=6a1d2f96-8cdf-4d1a-943d-7b73f4dfbb6d
